<a href="https://colab.research.google.com/github/Rakshithts/myprojects/blob/main/fine_tuned_model_for_news_format_sentence_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets peft accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.1 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
import os
import torch


In [ ]:
# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
# 1. Load a larger dataset (AG News)
raw_dataset = load_dataset("ag_news")
train_texts = raw_dataset["train"]["text"][:20000]

dataset = Dataset.from_dict({"text": train_texts})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [ ]:
# 2. Load tokenizer & model
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# 3. Tokenize with labels
def tokenize(batch):
    tokenized = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_ds = dataset.map(tokenize, batched=True, remove_columns=["text"])

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
# 4. Apply LoRA (PEFT)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["c_attn", "c_proj"],  # GPT-2 attention layers
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:2156: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
# 5. Training setup
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,   # bigger batch size for speed
    learning_rate=5e-4,
    num_train_epochs=5,
    logging_steps=50,
    report_to="none",
    save_strategy="no",
    fp16=torch.cuda.is_available()   # mixed precision if GPU supports it
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds
)


In [ ]:
# 6. Train
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.856100
100,3.518000
150,3.340800
200,3.207300
250,3.337900
300,3.203000
350,3.233000
400,3.208300
450,3.201300
500,3.110600


TrainOutput(global_step=25000, training_loss=2.8655124267578125, metrics={'train_runtime': 982.9798, 'train_samples_per_second': 101.731, 'train_steps_per_second': 25.433, 'total_flos': 1648676044800000.0, 'train_loss': 2.8655124267578125, 'epoch': 5.0})

In [ ]:
# 7. Test generation
prompt = "The weather today"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=30, do_sample=True, top_k=40, top_p=0.9)
print("\nGenerated text:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated text:
 The weather today has gotten better, and is going to get better. The temperature and humidity are not good, and not in a way that was predicted yesterday. But


In [ ]:
# 7. Test generation
prompt = "The movie kept the audience"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=30, do_sample=True, top_k=40, top_p=0.9)
print("\nGenerated text:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Generated text:
 The movie kept the audience's eye on it for the first time in a year.


In [ ]:
from transformers import AutoModelForCausalLM

# Function for generating text
def generate_text(model, tokenizer, prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=True,
        top_k=40,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = "The politics"

# 1. Load base DistilGPT-2 for BEFORE result
base_model = AutoModelForCausalLM.from_pretrained("distilgpt2").to(model.device)
base_model.eval()

print("=== BEFORE Fine-Tuning ===")
before_text = generate_text(base_model, tokenizer, prompt)
print(before_text)

# 2. AFTER: Fine-tuned model (already trained above)
print("\n=== AFTER Fine-Tuning ===")
after_text = generate_text(model, tokenizer, prompt)
print(after_text)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== BEFORE Fine-Tuning ===
The politics of the year is always changing, and it is a good thing for the American people to learn how to do it.‍

The last

=== AFTER Fine-Tuning ===
The politics of war in Iraq: Iraq #39;s first step of the war. From Iraq #39;s first step. From the US to Iraq
